# April 2018 Metrics
#### By Julien Dhouti
This is a notebook analyzing my performance during the month of April on the phone while I was working at GoDaddy.
The analysis analyzes sales, survey scores, conversation rates, and even customer availability.

While creating the graphs, I was looking for patterns and trends between different variables. I can use the insight found inside of this analysis to improve my performance on the phone during later months.

#### This notebook is split into 2 sections: the cleaning and the visualization.

The libraries I used for this notebook include:
* the one and only Pandas
* Plotly <3
* and Numpy

**Note: I later found out that I was not able to release the actual data so the data used in this notebook was replaced by randomly generated data.**

In [7]:
# First we import the necessary libraries
import pandas as pd
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import numpy as np

# turn off uploads for graphs in this notebook:
init_notebook_mode(connected=True)

In [8]:
# Now we can import the data so that we can start manipulating it
df = pd.read_csv('data/random_data.csv')
df = df.drop(['Unnamed: 0'], axis=1)

# Take a peak at it
df.head()

,date,cust_avail_v3,css_count,css_score,orders,new_conv,new_sales,new_sales_perc,sales,calls_per_day
0,2018-04-02,98.22999999999999%,2,9.60,2.0,9.790000000000001%,$927.3,73.26%,$1265.68,21.0
1,2018-04-03,90.2%,1,1.78,4.0,8.36%,$595.68,97.00999999999999%,$614.02,32.0
2,2018-04-04,95.47%,1,3.32,6.0,8.3%,$67.88,25.91%,$262.02,29.0
3,2018-04-05,86.78%,2,2.51,3.0,20.59%,$249.05,34.2%,$728.13,39.0
4,2018-04-09,102.35000000000001%,4,3.19,2.0,13.74%,$821.25,93.08%,$882.27,27.0


In [9]:
# Let's check out some details about the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 10 columns):
date              19 non-null object
cust_avail_v3     19 non-null object
css_count         19 non-null int64
css_score         19 non-null float64
orders            19 non-null float64
new_conv          19 non-null object
new_sales         19 non-null object
new_sales_perc    19 non-null object
sales             19 non-null object
calls_per_day     19 non-null float64
dtypes: float64(3), int64(1), object(6)
memory usage: 1.6+ KB


By the looks of it, it looks like I should edit the types of the columns. But first I want to get rid of the dollar signs and percentage signs in each of those columns. I can do this by creating a few functions that I will apply to the columns using the vectorized function `.apply()`.

In [10]:
def convert_dollar(value):
    """Will remove the dollar sign in the passed in string and convert it to a float."""
    value = value.strip()          # remove any trailing spaces
    value = value.replace(',', '') # remove commas
    new_value = value[1:]
    try:
        data = float(new_value)
    except:
        return 0
    return new_value

def convert_perc(perc):
    """Will remove the percentage sign in the passed in string and convert it to a float."""
    no_perc = perc[:-1]
    # convert to decimal form
    return float(no_perc) / 100

Now we can apply these functions to the dataframes that have need symbols removed.

In [11]:
# The columns with dollar signs
df['new_sales'] = df.new_sales.apply(convert_dollar)
df['sales'] = df.sales.apply(convert_dollar)

In [12]:
# The columns with percentages
df['cust_avail_v3'] = df.cust_avail_v3.apply(convert_perc)
df['new_conv'] = df.new_conv.apply(convert_perc)
df['new_sales_perc'] = df.new_sales_perc.apply(convert_perc)

Now that we have cleaned these columns, I take a look at them to make sure everything went well.

In [13]:
df

,date,cust_avail_v3,css_count,css_score,orders,new_conv,new_sales,new_sales_perc,sales,calls_per_day
0,2018-04-02,0.9823,2,9.60,2.0,0.0979,927.3,0.7326,1265.68,21.0
1,2018-04-03,0.9020,1,1.78,4.0,0.0836,595.68,0.9701,614.02,32.0
2,2018-04-04,0.9547,1,3.32,6.0,0.0830,67.88,0.2591,262.02,29.0
3,2018-04-05,0.8678,2,2.51,3.0,0.2059,249.05,0.3420,728.13,39.0
4,2018-04-09,1.0235,4,3.19,2.0,0.1374,821.25,0.9308,882.27,27.0
5,2018-04-10,0.8141,3,2.49,7.0,0.0058,433.79,0.4503,963.36,34.0
6,2018-04-11,0.9361,0,5.90,8.0,0.2049,147.79,0.4445,332.47,37.0
7,2018-04-12,0.8629,2,1.51,4.0,0.1214,440.37,0.7070,622.88,29.0
8,2018-04-13,0.9689,7,8.05,3.0,0.2054,227.79,0.4270,533.51,35.0
9,2018-04-16,0.8183,5,0.20,2.0,0.1977,693.73,0.6861,1011.15,30.0


Normally, this wouldn't be a very good tactic to monitor entire column changes but since the data happens to be so small, I can just look through the columns myself without creating a function.

In any case, the columns that we modified seem to have been modified correctly so we can move on to the next steps: convert columns to their appropriate datatype to save some memory.

In [14]:
# First we convert the 'date' column to a datetime column
df['date'] = pd.to_datetime(df['date'])

We were able to directly convert the date column because nothing was wrong in it but for other columns which have `NaN` values, I will have to deal with those first to prevent any errors.

The part that is even more tedious is that I have to deal with `NaN` values differently based on the columns. For example in the `new_sales` column, any missing data can be interpreted as $0 in sales so we would just replace any `NaN` value with 0.

In [15]:
# In the CSS score column, we will replace missing values with the 0 as well
df['css_score'] = pd.to_numeric(df.css_score, errors='coerce') # convert all values to float first
df['css_score'] = df.css_score.fillna(0)

df.head(7)

,date,cust_avail_v3,css_count,css_score,orders,new_conv,new_sales,new_sales_perc,sales,calls_per_day
0,2018-04-02,0.9823,2,9.60,2.0,0.0979,927.3,0.7326,1265.68,21.0
1,2018-04-03,0.9020,1,1.78,4.0,0.0836,595.68,0.9701,614.02,32.0
2,2018-04-04,0.9547,1,3.32,6.0,0.0830,67.88,0.2591,262.02,29.0
3,2018-04-05,0.8678,2,2.51,3.0,0.2059,249.05,0.3420,728.13,39.0
4,2018-04-09,1.0235,4,3.19,2.0,0.1374,821.25,0.9308,882.27,27.0
5,2018-04-10,0.8141,3,2.49,7.0,0.0058,433.79,0.4503,963.36,34.0
6,2018-04-11,0.9361,0,5.90,8.0,0.2049,147.79,0.4445,332.47,37.0


In [16]:
# Then we can convert all of the other columns to float64 type since they are all decimals
float_columns = df.columns[1:]

for column in float_columns:
    df[column].astype(np.float64)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 10 columns):
date              19 non-null datetime64[ns]
cust_avail_v3     19 non-null float64
css_count         19 non-null int64
css_score         19 non-null float64
orders            19 non-null float64
new_conv          19 non-null float64
new_sales         19 non-null object
new_sales_perc    19 non-null float64
sales             19 non-null object
calls_per_day     19 non-null float64
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 1.6+ KB


In [18]:
# We check once more for anymore missing values
df.isnull().all().all()

False

We can now export the data so that we can visualize it

In [19]:
df.to_csv('clean_random_data.csv')